## Modelagem preditiva com dados Out-of-Core utilizando Dask

Alunas: Ana Beatriz Parra Ferreira, Bruna Meinberg, Luana Abramoff 


### Dados Out-of-Core 

Dados out-of-core são conjuntos de dados que ultrapassam a capacidade de memória RAM disponível em uma máquina. Este termo é frequentemente utilizado no contexto de bancos de dados, sistemas de arquivos e computação científica, onde o manuseio eficiente de grandes volumes de dados é crítico. 

O principal desafio ao trabalhar com dados out-of-core é o gerenciamento eficiente do I/O (entrada e saída) de dados, uma vez que o acesso contínuo a um disco rígido ou a uma rede pode se tornar um gargalo significativo. Além disso, é fundamental manter a eficiência no processamento para evitar a deterioração do desempenho devido ao constante carregamento e descarregamento de dados.

A capacidade de processar dados out-of-core é crucial para muitas aplicações de ciência de dados, especialmente em um mundo onde o volume de dados continua a crescer exponencialmente. As ferramentas que suportam esses processos são essenciais para permitir análises complexas e insights que de outra forma não seriam possíveis.

### Estratégias de Processamento

Para processar dados out-of-core de maneira eficaz, as estratégias a seguir são comumente adotadas:

1. **Streaming de dados**: Os dados são processados sequencialmente em pequenos blocos, permitindo que operações sejam realizadas em cada bloco à medida que passam pela memória.
2. **Indexação eficiente**: Criar índices que permitem acessar rapidamente partes específicas dos dados sem a necessidade de carregar conjuntos de dados inteiros na memória.
3. **Algoritmos otimizados para out-of-core**: Utilizar ou desenvolver algoritmos que minimizam a necessidade de movimentação de dados entre o disco e a RAM.
4. **Parallel processing**: Paralelizar o processamento de dados tanto quanto possível para maximizar o uso de todos os núcleos disponíveis, tratando diferentes partes dos dados simultaneamente.

### Ferramentas e Tecnologias

Várias ferramentas e tecnologias foram desenvolvidas para facilitar o manuseio de dados out-of-core:

- **Dask**: Fornece estruturas de dados e uma programação paralela que permite trabalhar eficientemente com grandes conjuntos de dados de maneira simples e integrada.
- **Pandas com chunksize**: Pandas pode processar arquivos grandes em pedaços menores com o parâmetro `chunksize`, permitindo operações em partes do arquivo de cada vez.
- **Bibliotecas de aprendizado de máquina adaptadas**: Algumas bibliotecas de machine learning, como o `scikit-learn`, têm opções ou versões que suportam aprendizado incrementativo para dados grandes.


O seguinte notebook tem como objetivo mostrar de forma detalhada como fazer um modelo preditivo no seu computador para dados que nem cabem na RAM faazendo uso do Dask. 

-------





# Introdução ao Dask

Dask é uma biblioteca de computação paralela e distribuída que integra de maneira harmoniosa com o ecossistema do Python, principalmente com as bibliotecas Pandas, NumPy e Scikit-Learn. Desenvolvido para operar com conjuntos de dados que excedem a memória RAM disponível, o Dask permite análises complexas e modelos de aprendizado de máquina em grandes volumes de dados.

O Dask tornou-se uma ferramenta essencial no arsenal de cientistas de dados que enfrentam desafios associados ao manejo de grandes conjuntos de dados. Sua habilidade em se integrar de forma suave ao ambiente Python, junto com sua simplicidade de uso e poderoso desempenho, o destaca como uma escolha primordial para computação de dados em grande escala.

### Por que usar Dask?

A crescente disponibilidade de grandes conjuntos de dados pode ser um desafio considerável para os cientistas de dados. A limitação da memória RAM significa que dados extensos não podem ser processados diretamente. A solução tradicional para isso seria usar ferramentas como Spark ou Hadoop, que, embora eficazes, podem ser complexas e pesadas para configurar e operar. O Dask, por outro lado, oferece uma interface simples e flexível, que reduz a complexidade e se integra facilmente ao fluxo de trabalho Python existente.

## Principais Características do Dask

- **Escalabilidade**: Dask é escalável verticalmente e horizontalmente, ou seja, pode processar dados em um único computador usando seus múltiplos núcleos ou em um cluster de máquinas.
- **Flexibilidade**: Suporta diversas operações como agregações, joins, leitura de dados em diferentes formatos, além de algoritmos de aprendizado de máquina.
- **Desempenho**: Utiliza computação paralela e otimizações inteligentes para acelerar a execução.
- **Lazy Execution**: O Dask adia a execução das operações até que seja explicitamente solicitado. Isso permite a otimização e paralelização eficientes dos cálculos.
- **Integração**: Trabalha de maneira nativa com as bibliotecas de ciência de dados Python, como Pandas, NumPy e Scikit-Learn, facilitando a adoção por parte de quem já está familiarizado com esses pacotes.

## Aplicações do Dask

O Dask é ideal para cenários onde o volume de dados é grande demais para a memória de uma máquina única, mas pequeno demais para justificar a complexidade de um sistema de processamento de dados distribuídos, como Hadoop ou Spark. É comumente utilizado em:

- Análises exploratórias de dados
- Processamento e limpeza de grandes conjuntos de dados
- Treinamento de modelos de machine learning em grandes volumes de dados
- Simulações e modelagens que exigem alta capacidade computacional


### Documentação

Para mais informações, acessar a documentação da biblioteca disponível em: https://docs.dask.org/en/stable/dataframe.html

-------


# Modelo Preditivo 

Os dados utilizados para o seguinte modelo são do Registro Nacional de Acidentes e Estatísticas de Trânsito (Renaest), sob a
coordenação do Departamento Nacional de Trânsito (Denatran),que organiza e junta os dados dos Detrans de
cada unidade federativa. Os dados desta base são alimentados pelos boletins de ocorrência da polícia.


## 1. Instalação e configuração 

Para instalar o Dask e outras bibliotecas necessárias, é necessário rodar o seguinte comando: 

```

!pip install dask[complete] 

```

O Dask tem um comportamento muito similar ao Pandas. De mesmo modo, o Dask.Array é analogo á biblioteca Numpy. 
Para utilizá-los após a instalação, basta importar as bibliotecas necessárias: 

In [4]:
import dask.dataframe as dd # import pandas as pd
import dask.array as da # import numpy as np
import matplotlib.pyplot as plt

import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyRegressor

Importanto os dataframes que serão utilizados neste projeto: 

In [5]:
acidentes = dd.read_csv('dados/Acidentes_DadosAbertos_20230812.csv', blocksize=25e6, delimiter=';', assume_missing=True) # 25MB chunks  [manter suas partições com tamanho inferior a 100 MB]
localidade = dd.read_csv('dados/Localidade_DadosAbertos_20230812.csv', blocksize=25e6, delimiter=';', assume_missing=True) # 25MB chunks  [manter suas partições com tamanho inferior a 100 MB]
veiculo = dd.read_csv('dados/TipoVeiculo_DadosAbertos_20230812.csv', blocksize=25e6, delimiter=';', assume_missing=True) # 25MB chunks  [manter suas partições com tamanho inferior a 100 MB]

##### Realizando o merge das bases.
Para juntar a base 'acidentes' com 'localidade' a chave 'chv_localidade' será utilizada. Para concatenar com as informações de veículos, a chave 'num_acidente', que se refere ao número do boletim de ocorrência, será utilizada.

(Aviso: vai demorar um pouco)

In [6]:
#merge dos dfs 
result = dd.merge(acidentes, localidade, on='chv_localidade', how='inner')
result = dd.merge(result, veiculo, on='num_acidente', how='inner')
# result = dd.merge(result, vitimas, on='chv_localidade', how='inner')
df_final = result.compute()


c:\Users\anapa\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\anapa\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\anapa\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\anapa\AppData\Local\Programs\Python\Python311\Lib\site-packages\dask\dataframe\io\csv.py:195: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = reader(bio, **kwargs)
c:\Users\anapa\AppData\Local\Programs\Python\Python311\Lib\site-pack

In [7]:
df_final.head(10)

,num_acidente,chv_localidade,data_acidente,uf_acidente,ano_acidente,mes_acidente,mes_ano_acidente,codigo_ibge_x,dia_semana,fase_dia,...,uf,codigo_ibge_y,municipio,regiao_metropolitana,qtde_habitantes,frota_total,frota_circulante,tipo_veiculo,ind_veic_estrangeiro,qtde_veiculos
0,4267595.0,SP3543402202109,2021-09-03,SP,2021.0,9.0,92021.0,3543402.0,SEXTA-FEIRA,NOITE,...,SP,3543402.0,RIBEIRAO PRETO,nao,717721.0,543447.0,355794.0,CAMINHONETE,NAO INFORMADO,1.0
1,4267595.0,SP3543402202109,2021-09-03,SP,2021.0,9.0,92021.0,3543402.0,SEXTA-FEIRA,NOITE,...,SP,3543402.0,RIBEIRAO PRETO,nao,717721.0,543447.0,355794.0,MOTOCICLETA,NAO INFORMADO,1.0
2,340180.0,SP3524402202109,2021-09-22,SP,2021.0,9.0,92021.0,3524402.0,QUARTA-FEIRA,TARDE,...,SP,3524402.0,JACAREI,nao,236642.0,150619.0,94753.0,MOTOCICLETA,NAO INFORMADO,1.0
3,3639704.0,SP3509502202109,2021-09-04,SP,2021.0,9.0,92021.0,3509502.0,SABADO,TARDE,...,SP,3509502.0,CAMPINAS,nao,1220576.0,911417.0,545585.0,AUTOMOVEL,NAO INFORMADO,1.0
4,1054524.0,SP3507506202109,2021-09-18,SP,2021.0,9.0,92021.0,3507506.0,SABADO,MANHA,...,SP,3507506.0,BOTUCATU,nao,149261.0,108288.0,68650.0,MOTOCICLETA,NAO INFORMADO,1.0
5,1054524.0,SP3507506202109,2021-09-18,SP,2021.0,9.0,92021.0,3507506.0,SABADO,MANHA,...,SP,3507506.0,BOTUCATU,nao,149261.0,108288.0,68650.0,CAMINHAO,NAO INFORMADO,1.0
6,2378247.0,SP3541000202109,2021-09-24,SP,2021.0,9.0,92021.0,3541000.0,SEXTA-FEIRA,NOITE,...,SP,3541000.0,PRAIA GRANDE,sim,334808.0,163739.0,115814.0,MOTOCICLETA,NAO INFORMADO,1.0
7,3363031.0,SP3516200202109,2021-09-08,SP,2021.0,9.0,92021.0,3516200.0,QUARTA-FEIRA,MANHA,...,SP,3516200.0,FRANCA,nao,357799.0,281839.0,176840.0,AUTOMOVEL,NAO INFORMADO,2.0
8,893065.0,SP3549805202109,2021-09-16,SP,2021.0,9.0,92021.0,3549805.0,QUINTA-FEIRA,MADRUGADA,...,SP,3549805.0,SAO JOSE DO RIO PRETO,nao,467980.0,406346.0,266561.0,AUTOMOVEL,NAO INFORMADO,1.0
9,967248.0,SP3552502202109,2021-09-01,SP,2021.0,9.0,92021.0,3552502.0,QUARTA-FEIRA,TARDE,...,SP,3552502.0,SUZANO,sim,302587.0,157207.0,99038.0,MOTOCICLETA,NAO INFORMADO,1.0


In [8]:
# # Verificando a quantidade de valores faltantes por coluna
# df_final.isnull().sum()

Neste projeto, a variável de interesse será uma categoria de acidente. A partir da varíavel tipo deacidente (tp_acidente), uma coluna Dummy de acidentes do tipo COLISÃO LATERAL. 

Para isso, a função dask.bag.Bag.map_partitions da bibliteca Dask será utilizada:


In [9]:
df_final['tp_acidente'].value_counts()

tp_acidente
COLISAO                                      1040900
DESCONHECIDO                                 1032954
COLISAO LATERAL                               704684
CHOQUE                                        567707
COLISAO TRASEIRA                              491800
COLISAO FRONTAL                               397790
OUTROS ACIDENTES DE TRANSITO                  390104
COLISAO TRANSVERSAL                           322673
QUEDA                                         202576
NAO INFORMADO                                 171170
ATROPELAMENTO COM PEDESTRE                    130420
TOMBAMENTO                                     60921
CAPOTAMENTO                                    59590
ATROPELAMENTO COM ANIMAIS                      49981
ENGAVETAMENTO                                  42403
ACIDENTE PESSOAL DE TRANSITO COM PEDESTRE        616
Name: count, dtype: int64

In [10]:
df_final['colisao_lateral'] = df_final['tp_acidente'] == 'COLISÃO LATERAL'
df_final.head()


,num_acidente,chv_localidade,data_acidente,uf_acidente,ano_acidente,mes_acidente,mes_ano_acidente,codigo_ibge_x,dia_semana,fase_dia,...,codigo_ibge_y,municipio,regiao_metropolitana,qtde_habitantes,frota_total,frota_circulante,tipo_veiculo,ind_veic_estrangeiro,qtde_veiculos,colisao_lateral
0,4267595.0,SP3543402202109,2021-09-03,SP,2021.0,9.0,92021.0,3543402.0,SEXTA-FEIRA,NOITE,...,3543402.0,RIBEIRAO PRETO,nao,717721.0,543447.0,355794.0,CAMINHONETE,NAO INFORMADO,1.0,False
1,4267595.0,SP3543402202109,2021-09-03,SP,2021.0,9.0,92021.0,3543402.0,SEXTA-FEIRA,NOITE,...,3543402.0,RIBEIRAO PRETO,nao,717721.0,543447.0,355794.0,MOTOCICLETA,NAO INFORMADO,1.0,False
2,340180.0,SP3524402202109,2021-09-22,SP,2021.0,9.0,92021.0,3524402.0,QUARTA-FEIRA,TARDE,...,3524402.0,JACAREI,nao,236642.0,150619.0,94753.0,MOTOCICLETA,NAO INFORMADO,1.0,False
3,3639704.0,SP3509502202109,2021-09-04,SP,2021.0,9.0,92021.0,3509502.0,SABADO,TARDE,...,3509502.0,CAMPINAS,nao,1220576.0,911417.0,545585.0,AUTOMOVEL,NAO INFORMADO,1.0,False
4,1054524.0,SP3507506202109,2021-09-18,SP,2021.0,9.0,92021.0,3507506.0,SABADO,MANHA,...,3507506.0,BOTUCATU,nao,149261.0,108288.0,68650.0,MOTOCICLETA,NAO INFORMADO,1.0,False


In [11]:
# Filtrando apenas colunas numéricas
numeric_cols = df_final.select_dtypes(include=['float64', 'int64'])
# Calculando correlação
corr_matrix = numeric_cols.corr() 
corr_matrix

,num_acidente,ano_acidente,mes_acidente,mes_ano_acidente,codigo_ibge_x,num_end_acidente,cep_acidente,km_via_acidente,latitude_acidente,longitude_acidente,...,qtde_feridosilesos,qtde_obitos,ano_referencia,mes_referencia,mes_ano_referencia,codigo_ibge_y,qtde_habitantes,frota_total,frota_circulante,qtde_veiculos
num_acidente,1.000000,0.001169,0.000649,0.000649,0.000349,-0.000377,0.000403,-0.000132,0.002175,0.001485,...,-0.000473,-0.000213,0.001169,0.000649,0.000649,0.000349,-0.000279,0.000005,-0.000098,-0.000086
ano_acidente,0.001169,1.000000,-0.111392,-0.111350,0.115954,0.096383,0.083975,0.022095,0.070777,0.051901,...,0.014077,-0.024498,1.000000,-0.111392,-0.111350,0.115954,0.026772,0.070151,0.100338,0.041328
mes_acidente,0.000649,-0.111392,1.000000,1.000000,0.017652,-0.008796,-0.000421,-0.012571,-0.008924,-0.007017,...,-0.019259,0.004322,-0.111392,1.000000,1.000000,0.017652,-0.015770,-0.011680,-0.010001,-0.001678
mes_ano_acidente,0.000649,-0.111350,1.000000,1.000000,0.017657,-0.008792,-0.000417,-0.012570,-0.008922,-0.007015,...,-0.019258,0.004321,-0.111350,1.000000,1.000000,0.017657,-0.015769,-0.011677,-0.009996,-0.001676
codigo_ibge_x,0.000349,0.115954,0.017652,0.017657,1.000000,0.197048,-0.022077,-0.029063,0.125570,0.092083,...,0.030575,-0.053096,0.115954,0.017652,0.017657,1.000000,0.030395,0.069528,0.090291,-0.027828
num_end_acidente,-0.000377,0.096383,-0.008796,-0.008792,0.197048,1.000000,-0.009028,-0.002775,0.016751,0.012285,...,0.017448,-0.018389,0.096383,-0.008796,-0.008792,0.197048,0.152400,0.143831,0.193733,0.045613
cep_acidente,0.000403,0.083975,-0.000421,-0.000417,-0.022077,-0.009028,1.000000,-0.004517,0.018388,0.013486,...,0.008356,-0.007309,0.083975,-0.000421,-0.000417,-0.022077,0.022204,-0.002978,0.010004,0.019646
km_via_acidente,-0.000132,0.022095,-0.012571,-0.012570,-0.029063,-0.002775,-0.004517,1.000000,0.005738,0.004208,...,0.007411,0.018451,0.022095,-0.012571,-0.012570,-0.029063,-0.004776,-0.003260,-0.000368,0.010309
latitude_acidente,0.002175,0.070777,-0.008924,-0.008922,0.125570,0.016751,0.018388,0.005738,1.000000,0.743361,...,-0.014740,-0.014110,0.070777,-0.008924,-0.008922,0.125570,0.035605,0.036027,0.039609,0.013755
longitude_acidente,0.001485,0.051901,-0.007017,-0.007015,0.092083,0.012285,0.013486,0.004208,0.743361,1.000000,...,-0.010345,-0.008119,0.051901,-0.007017,-0.007015,0.092083,0.023043,0.024114,0.026387,0.011029


## Limpeza dos dados e transformação de variáveis

In [12]:
df_final.shape

(5666289, 50)

In [13]:
# Pegandos todas as colunas
# print(df_final.dtypes[:20])
# print(df_final.dtypes[20:40])
# print(df_final.dtypes[40:60])

In [14]:
categorical_columns = [
'chv_localidade',
'data_acidente',
'uf_acidente',
'dia_semana',
'fase_dia',
'tp_acidente',
'cond_meteorologica',
'end_acidente',
'bairro_acidente',
'tp_rodovia',
'cond_pista',
'tp_cruzamento',
'tp_pavimento',
'tp_curva',
'lim_velocidade',
'tp_pista',
'ind_guardrail',
'ind_cantcentral',
'ind_acostamento',
'regiao',
'uf',
'municipio',
'regiao_metropolitana',
'tipo_veiculo',
'ind_veic_estrangeiro',
'colisao_lateral',
]

numerical_columns = [  
'num_acidente',
'ano_acidente',
'mes_acidente',
'mes_ano_acidente',
'codigo_ibge_x',
'num_end_acidente',
'cep_acidente',
'km_via_acidente',
'latitude_acidente',
'longitude_acidente',
'hora_acidente',
'qtde_acidente',
'qtde_acid_com_obitos',
'qtde_envolvidos',
'qtde_feridosilesos',
'qtde_obitos',
'ano_referencia',
'mes_referencia',
'mes_ano_referencia',
'codigo_ibge_y',
'qtde_habitantes',
'frota_total',
'frota_circulante',
'qtde_veiculos',
]

for column_group, column_type in (
    (categorical_columns, 'category'),
    (numerical_columns, 'float64'),
    #(ordinal_columns, 'int64'),
):
    for column in column_group:
        df_final[column] = df_final[column].astype(column_type)

# df_final.dtypes

In [21]:
# Filtrando apenas as colunas com valor nulo
null_counts = df_final.isnull().sum()
print(null_counts[null_counts > 0])

end_acidente           205914
bairro_acidente       2456511
latitude_acidente     4367708
longitude_acidente    4367843
dtype: int64


Avaliando a quantidade de valores nulos, percebe-se que as colunas end_acidente, bairro_acidente, latitude_acidente, longitude_acidente possuem uma grande quantidade ausente. 

| Variável     | qtd nulo (%)  
|--------------|--------------|
| end_acidente| 0.036 |
| bairro_acidente| 0.43 | 
| latitude_acidente | 0.77 | 
| longitude_acidente |  0.77|

Por mais que valores como longitude e latidude sejam muito importantes para uma avaliação sobre acidentes, a proporção de valores nulos de 77% é muito grande. Assim, os valores serão excluídos da análise, juntamente com bairro_acidente. Felizmente, a base do RENAEST possue outras variáveis de localidade que proporcionarão uma boa análise do local, compensando, portanto, a exclusão dos dados de longitude e latitude. 


In [15]:
#Como os a quantidade de nulos em end_acidente é 3% da base e possuimos muitos (!) dados, realizaremos um dropna() 
df_final = df_final.dropna(subset=['end_acidente'])

In [43]:
df_final.shape

(5460375, 50)

Fazendo uma limpeza nas colunas:

In [16]:
lista_drop = [
'latitude_acidente', #muitos valores nulos
'longitude_acidente', #muitos valores nulos
'bairro_acidente', #muitos valores nulos
'frota_total', # frota circulante foi considerado mais relevante para a análise
'codigo_ibge_x',
'codigo_ibge_y', 
'chv_localidade',
'data_acidente', # vamos usar o mes, dia da semana e período do dia
'tp_acidente', # fonte da nossa variavel de interesse
'uf_acidente', #igual a 'uf' que será utilizada
'tp_cruzamento', # muitos 'NAO INFORMADO'
'tp_curva', # muitos 'NAO INFORMADO'
'lim_velocidade', # muitos 'NAO INFORMADO'
'tp_pista', # muitos 'NAO INFORMADO'
'ind_veic_estrangeiro',
'regiao',  # ja usando informações sobre ufs
'ind_acostamento', # muitos 'NAO INFORMADO'
'ind_cantcentral', # muitos 'NAO INFORMADO'
'ind_guardrail', # muitos 'NAO INFORMADO'
'mes_ano_referencia', # muitos 'NAO INFORMADO'
'qtde_acidente', # só valores 1 
'fase_dia',   # vamos usar hora do acidente
'num_end_acidente',
'cep_acidente', 
'km_via_acidente', 
'num_acidente'  #boletim de ocorrencia
]

df_final.drop(columns=lista_drop, inplace=True)

## Separação treino-teste e modelagem inicial

A biblioteca Dask disponibiliza uma função de split treino-teste muito semelhante ao scikit-learn.

In [17]:
# Split features and target, and return.
X = df_final.drop(columns=['colisao_lateral']).copy()
y = df_final['colisao_lateral'].copy()

In [19]:
# no sklearn: from sklearn.model_selection import train_test_split 
from dask_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    random_state=42,
    shuffle=True
    )

## Definição do modelo 

In [20]:
import dask_ml
from dask_ml.model_selection import GridSearchCV , ShuffleSplit
from dask_ml.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from dask_ml.preprocessing import StandardScaler
from dask_ml.impute import SimpleImputer
from dask_ml.preprocessing import OneHotEncoder

In [23]:
categorical_features = X.select_dtypes(include=['category']).columns #variáveis categóricas
numerical_features = X.select_dtypes(include=['float64', 'int64']).columns #variáveis numéricas

num_pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('imputer', SimpleImputer(strategy='mean')),
])

cat_pipeline = Pipeline([
    ('encoder', OneHotEncoder()),    #drop='first')),
])

preprocessing_pipeline = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, numerical_features),
        ('cat', cat_pipeline, categorical_features),
    ],
    remainder='passthrough',
)

pipe = Pipeline([
    ('preprocessor', preprocessing_pipeline),
    ('lasso', LinearRegression(penalty='l1')),
])

pipe

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler()),
                                                                  ('imputer',
                                                                   SimpleImputer())]),
                                                  Index(['ano_acidente', 'mes_acidente', 'mes_ano_acidente', 'hora_acidente',
       'qtde_acid_com_obitos', 'qtde_envolvidos', 'qtde_feridosilesos',
       'qtde_obitos', 'ano_referencia', 'mes_referencia', 'qtde_habitantes',
       'frota_circulante', 'qtde_veiculos'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['dia_semana', 'cond_meteorologica', 'end_acidente', 'tp_rodovia',
       'cond_pista', 'tp_pavimento', 'uf', 'municipio', 'regiao_metropolitana',
       'tipo_veiculo'],
      dtype='object'))])),
                ('lasso', LinearRegression(penalty='l1'))])

In [27]:
# # Quanto maior o numero de splits, maior a significância estatística da
# # validação cruzada, mas também maior o tempo de execução.
# # num_splits = 100

# num_splits = 50

# param_grid = [
#     {
#         # 'regressor': [Lasso()],
#         'regressor__alpha': [0.01, 0.1, 1, 10, 100, 500, 1000],
#     },
# ]

# test_fraction = 0.2
# num_samples_total = len(y_train)
# num_samples_test = int(test_fraction * num_samples_total)
# num_samples_train = num_samples_total - num_samples_test

# grid = GridSearchCV( #para encontrar os melhores parâmetros usando validação cruzada
#     pipe,
#     param_grid,
#     cv=ShuffleSplit(
#         n_splits=num_splits,
#         test_size=num_samples_test,
#         random_state=42,
#     ),
#     n_jobs=-1,
#     scoring='neg_root_mean_squared_error',
# )
# grid.fit(X_train, y_train)


In [28]:
#  num_splits = 100

# param_grid = [
#     {
#         'regressor': [Lasso()],
#         'regressor__alpha': [0.01, 0.1, 1, 10, 100, 500, 1000],
#     },
# ]
# test_fraction = 0.2
# num_samples_total = len(y_train)
# num_samples_test = int(test_fraction * num_samples_total)
# num_samples_train = num_samples_total - num_samples_test
# grid = GridSearchCV( #para encontrar os melhores parâmetros usando validação cruzada
#    pipe,
#     param_grid,
#     cv=ShuffleSplit(
#         n_splits=num_splits,
#         test_size=num_samples_test,
#         random_state=42,
#     ),
#     n_jobs=-1,
#     scoring='neg_root_mean_squared_error',
# )
# grid.fit(X_train, y_train)

In [29]:
# # # Criação do Pipeline
# # pipeline = make_pipeline(
# #     StandardScaler(),  # Normalização das features
# #     LinearRegression(penalty='l1')  # Modelo Lasso
# # )

# # Grid de hiperparâmetros
# param_grid = {
#     'linearregression__alpha': [0.001, 0.01, 0.1, 1, 10]  # Alpha é o parâmetro de regularização
# }

# # Configuração do GridSearchCV
# grid_search = GridSearchCV(pipe, param_grid, cv=5)

# # Ajuste do Pipeline
# grid_search.fit(X, y)

# # Melhores parâmetros e melhor score
# print("Melhores parâmetros:", grid_search.best_params_)
# print("Melhor score:", grid_search.best_score_)
